In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
import seaborn as sns
import pandas_profiling

In [2]:
train_df = pd.read_csv('Dataset/train.csv')
soln_df = pd.read_csv('Dataset/test.csv')

In [3]:
train_df.drop(['pet_id'], axis = 1, inplace = True)
main_test = soln_df.drop(['pet_id'], axis = 1)

In [4]:
def pre_process(df):
    df.condition.fillna(3, inplace=True)
    df['time_to_shelter'] = (pd.to_datetime(df.listing_date) - pd.to_datetime(df.issue_date)).dt.days
    df.drop(['issue_date','listing_date'], axis = 1, inplace=True)
    
    top_colors = [
                     'Black',
                     'White',
                     'Brown',
                     'Brown Tabby',
                     'Tan',
                     'Blue',
                     'Orange Tabby',
                     'Red',
                     'Brown Brindle',
                     'Tricolor',
                     'Blue Tabby',
                     'Tortie',
                     'Calico',
                     'Gray',
                     'Chocolate',
                     'Torbie',
                     'Cream Tabby',
                     'Sable',
                     'Cream',
                     'Fawn',
                     'Yellow',
                     'Buff',
                     'Lynx Point',
                     'Blue Merle'
                ]
    
    def reduce_color(color):
        if color in top_colors:
            return color
        return 'Other'
    
    df['color_mod'] = df.color_type.apply(reduce_color)
    df.drop(['color_type'], axis = 1, inplace=True)
    df.drop(['length(m)','height(cm)'], axis = 1, inplace= True)
    
    return df

In [5]:
train_df = pre_process(train_df)
main_test = pre_process(main_test)

In [6]:
train_y1 = train_df.loc[:,['breed_category']]
train_y2 = train_df.loc[:,['pet_category']]

In [7]:
train_X = train_df.drop(['breed_category','pet_category'], axis=1)

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [9]:
ct = ColumnTransformer([("color", OneHotEncoder(sparse=False, drop='first'), [4])], remainder = 'passthrough')
ct.fit(train_X)

ColumnTransformer(n_jobs=None, remainder='passthrough', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('color',
                                 OneHotEncoder(categories='auto', drop='first',
                                               dtype=<class 'numpy.float64'>,
                                               handle_unknown='error',
                                               sparse=False),
                                 [4])],
                  verbose=False)

In [10]:
train_X = ct.transform(train_X)
main_test = ct.transform(main_test)

In [11]:
print(train_X.shape)
print(main_test.shape)

(18834, 28)
(8072, 28)


In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

sc = scaler.fit(train_X)

train_X = sc.transform(train_X)

main_test = sc.transform(main_test)

In [20]:
from sklearn.model_selection import train_test_split

X_train_breed, X_test_breed, y_train_breed, y_test_breed = train_test_split(train_X, train_y1, test_size = 0.2)
X_train_pet, X_test_pet, y_train_pet, y_test_pet = train_test_split(train_X, train_y2, test_size = 0.2)

In [21]:
from imblearn.over_sampling import RandomOverSampler

os = RandomOverSampler()

X_train_breed, y_train_breed = os.fit_resample(X_train_breed, y_train_breed)
X_train_pet, y_train_pet = os.fit_resample(X_train_pet, y_train_pet)

In [32]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix 

In [34]:
breed_model = SVC()

breed_model.fit(X_train_breed, y_train_breed)

breed_test_preds = breed_model.predict(X_test_breed) 

confusion_matrix(y_test_breed, breed_test_preds) 

/Users/parth/miniconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[1780,    8,    0],
       [ 367, 1315,    0],
       [   1,    0,  296]])

In [35]:
pet_model = SVC()

pet_model.fit(X_train_pet, y_train_pet)

pet_test_preds = pet_model.predict(X_test_pet) 

confusion_matrix(y_test_pet, pet_test_preds) 

/Users/parth/miniconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([[  13,    2,    1,    5],
       [  38, 1167,  243,   30],
       [  66,  177, 1785,   58],
       [  28,    6,   10,  138]])

In [36]:
breed_preds = breed_model.predict(main_test)

In [37]:
pet_preds = pet_model.predict(main_test)

In [38]:
submission = pd.DataFrame()

In [39]:
submission['pet_id'] = soln_df.pet_id

In [40]:
submission['breed_category'] = breed_preds

In [41]:
submission['pet_category'] = pet_preds

In [42]:
submission.breed_category = submission.breed_category.astype('int64')

In [43]:
submission

,pet_id,breed_category,pet_category
0,ANSL_75005,1,2
1,ANSL_76663,0,1
2,ANSL_58259,0,2
3,ANSL_67171,0,2
4,ANSL_72871,0,2
...,...,...,...
8067,ANSL_66809,0,0
8068,ANSL_59041,1,2
8069,ANSL_60034,1,2
8070,ANSL_58066,2,4


In [44]:
submission.to_csv('submission_4.csv',index=False)